# 4.0 - Análisis de sentimiento

Aqui realizar un análisis de sentimiento, intentando crear variables que me ayuden a explicar el precio del airbnb.

In [1]:
# librerias

import pandas as pd
pd.set_option('display.max_columns', None)

from googletrans import Translator

from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification

from catboost import CatBoostRegressor as CTR

from sklearn.model_selection import train_test_split as tts 

from sklearn.metrics import mean_squared_error as mse 
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import r2_score as r2

import warnings
warnings.filterwarnings('ignore')

In [2]:
reviews=pd.read_csv('../data/raw_data/reviews.csv.gz', compression='gzip', low_memory=False)

reviews.sort_values(by='date', ascending=False, inplace=True)

reviews.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
663914,24692982,590993861,2020-01-10,166362284,Chenkai,highly recommend to everyone!\nThe apartment i...
771650,33372756,590980346,2020-01-10,36962899,Alejandro,Buen alojamiento
756106,31958307,590993867,2020-01-10,157306680,Alberto,Me encanto el servicio y el trato. Volvere nue...
809859,39648780,591013976,2020-01-10,312328066,Yoshihiro,It was a wonderful experience for me. Geira co...
801351,37377619,591032285,2020-01-10,100337449,Elsa,"Volveremos!! Nos sentimos como en casa, buena ..."


In [3]:
reviews.shape

(812726, 6)

In [44]:
listings=pd.read_csv('../data/transform_data/listings_normal.csv')

listings=listings[(listings.price>=10) & (listings.price<=196)]  

listings.head()

,id,host_id,host_is_superhost,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,accommodates,bathrooms,bedrooms,beds,price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,x,y,z,property_type_apartment,property_type_barn,property_type_bed_and_breakfast,property_type_boutique_hotel,property_type_bungalow,property_type_camper_rv,property_type_casa_particular_cuba,property_type_cave,property_type_chalet,property_type_condominium,property_type_dome_house,property_type_earth_house,property_type_farm_stay,property_type_guest_suite,property_type_guesthouse,property_type_hostel,property_type_hotel,property_type_house,property_type_hut,property_type_igloo,property_type_loft,property_type_nature_lodge,property_type_other,property_type_pension_south_korea,property_type_serviced_apartment,property_type_tent,property_type_tiny_house,property_type_townhouse,property_type_villa,property_type_yurt,room_type_hotel_room,room_type_private_room,room_type_shared_room,bed_type_couch,bed_type_futon,bed_type_pull_out_sofa,bed_type_real_bed,memory_foam_mattress,beach_essentials,smart_tv,body_soap,day_bed,internet,standing_valet,tv,dryer,toilet,gas_oven,shower_chair,dogs,safety_card,wine_cooler,disabled_parking_spot,step_free_shower,wide_entrance,cooking_basics,formal_dining_area,hbo_go,long_term_stays_allowed,en_suite_bathroom,sun_loungers,beach_view,pocket_wifi,netflix,projector_and_screen,oven,balcony,free_street_parking,ethernet_connection,extra_space_around_bed,accessible_height_toilet,baby_bath,sound_system,patio_or_balcony,high_chair,fixed_grab_bars_for_toilet,first_aid_kit,essentials,toilet_paper,suitable_for_events,wide_entryway,ev_charger,mudroom,coffee_maker,garden_or_backyard,hot_water,family_kid_friendly,private_bathroom,paid_parking_on_premises,changing_table,handheld_shower_head,single_level_home,baby_monitor,wide_clearance_to_shower,washer,touchless_faucets,pets_allowed,crib,other,jetted_tub,steam_oven,host_greets_you,trash_can,laptop_friendly_workspace,private_living_room,buzzer_wireless_intercom,refrigerator,luggage_dropoff_allowed,full_kitchen,smoke_detector,soaking_tub,heated_floors,mobile_hoist,indoor_fireplace,bedroom_comforts,central_air_conditioning,hot_tub,cable_tv,heating,washer___dryer,well_lit_path_to_entrance,pets_live_on_this_property,iron,bathroom_essentials,high_resolution_computer_monitor,hangers,walk_in_shower,elevator,no_stairs_or_steps_to_enter,cats,game_console,extra_pillows_and_blankets,ski_in_ski_out,room_darkening_shades,firm_mattress,fireplace_guards,lake_access,warming_drawer,kitchenette,flat_path_to_guest_entrance,bbq_grill,bidet,air_conditioning,carbon_monoxide_detector,breakfast_table,dvd_player,cleaning_before_checkout,window_guards,kitchen,stair_gates,amazon_echo,outlet_covers,wide_hallways,fixed_grab_bars_for_shower,babysitter_recommendations,table_corner_guards,pack_n_play_travel_crib,gym,wide_entrance_for_guests,childrens_books_and_toys,keypad,outdoor_seating,printer,rain_shower,waterfront,hot_water_kettle,wifi,smart_lock,bath_towel,mini_fridge,convection_oven,terrace,beachfront,free_parking_on_premises,pillow_top_mattress,ceiling_fan,exercise_equipment,childrens_dinnerware,pool,breakfast,dishwasher,bed_linens,self_check_in,hammock,lock_on_bedroom_door,shampoo,double_oven,barbecue_utensils,microwave,other_pets,building_staff,bread_maker,lockbox,shower_gel,fire_extinguisher,paid_parking_off_premises,stove,hair_dryer,24_hour_check_in,ground_floor_access,doorman,smoking_allowed,wheelchair_accessible,accessible_height_bed,dishes_and_silverware,private_entrance,baking_sheet,bathtub,wide_doorway_to_guest_bathroom,heated_towel_rack,shared_pool,espresso_machine,bathtub_with_bath_chair
0,

In [45]:
listings.shape

(18936, 244)

In [46]:
# group by id, solo la primera review

primera=reviews.groupby('listing_id').first().reset_index()

primera.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,6369,576393686,2019-12-13,310436091,Enni,Stayed at Simon's and Arturo's place for two w...
1,21853,290844505,2018-07-15,49900000,Enrique,"Abdel es muy buen anfitrión, está atento y dis..."
2,24805,587542375,2020-01-03,317064017,Leila,"Trés bon emplacement, tout est à proximité, ir..."
3,24836,574437364,2019-12-08,69482035,Miguel,"La casa es mucho más bonita que en las fotos, ..."
4,26825,571630792,2019-12-01,91328885,Sooma,"Augstina was very nice and helpful, she has Ni..."


In [47]:
# todos los datos

total=listings.merge(primera, left_on='id', right_on='listing_id')

total=total.dropna()

total.drop(columns=['listing_id', 'id_y', 'date', 'reviewer_id', 'reviewer_name'])

total.reset_index(drop=True, inplace=True)

total.head()

,id_x,host_id,host_is_superhost,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,accommodates,bathrooms,bedrooms,beds,price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,x,y,z,property_type_apartment,property_type_barn,property_type_bed_and_breakfast,property_type_boutique_hotel,property_type_bungalow,property_type_camper_rv,property_type_casa_particular_cuba,property_type_cave,property_type_chalet,property_type_condominium,property_type_dome_house,property_type_earth_house,property_type_farm_stay,property_type_guest_suite,property_type_guesthouse,property_type_hostel,property_type_hotel,property_type_house,property_type_hut,property_type_igloo,property_type_loft,property_type_nature_lodge,property_type_other,property_type_pension_south_korea,property_type_serviced_apartment,property_type_tent,property_type_tiny_house,property_type_townhouse,property_type_villa,property_type_yurt,room_type_hotel_room,room_type_private_room,room_type_shared_room,bed_type_couch,bed_type_futon,bed_type_pull_out_sofa,bed_type_real_bed,memory_foam_mattress,beach_essentials,smart_tv,body_soap,day_bed,internet,standing_valet,tv,dryer,toilet,gas_oven,shower_chair,dogs,safety_card,wine_cooler,disabled_parking_spot,step_free_shower,wide_entrance,cooking_basics,formal_dining_area,hbo_go,long_term_stays_allowed,en_suite_bathroom,sun_loungers,beach_view,pocket_wifi,netflix,projector_and_screen,oven,balcony,free_street_parking,ethernet_connection,extra_space_around_bed,accessible_height_toilet,baby_bath,sound_system,patio_or_balcony,high_chair,fixed_grab_bars_for_toilet,first_aid_kit,essentials,toilet_paper,suitable_for_events,wide_entryway,ev_charger,mudroom,coffee_maker,garden_or_backyard,hot_water,family_kid_friendly,private_bathroom,paid_parking_on_premises,changing_table,handheld_shower_head,single_level_home,baby_monitor,wide_clearance_to_shower,washer,touchless_faucets,pets_allowed,crib,other,jetted_tub,steam_oven,host_greets_you,trash_can,laptop_friendly_workspace,private_living_room,buzzer_wireless_intercom,refrigerator,luggage_dropoff_allowed,full_kitchen,smoke_detector,soaking_tub,heated_floors,mobile_hoist,indoor_fireplace,bedroom_comforts,central_air_conditioning,hot_tub,cable_tv,heating,washer___dryer,well_lit_path_to_entrance,pets_live_on_this_property,iron,bathroom_essentials,high_resolution_computer_monitor,hangers,walk_in_shower,elevator,no_stairs_or_steps_to_enter,cats,game_console,extra_pillows_and_blankets,ski_in_ski_out,room_darkening_shades,firm_mattress,fireplace_guards,lake_access,warming_drawer,kitchenette,flat_path_to_guest_entrance,bbq_grill,bidet,air_conditioning,carbon_monoxide_detector,breakfast_table,dvd_player,cleaning_before_checkout,window_guards,kitchen,stair_gates,amazon_echo,outlet_covers,wide_hallways,fixed_grab_bars_for_shower,babysitter_recommendations,table_corner_guards,pack_n_play_travel_crib,gym,wide_entrance_for_guests,childrens_books_and_toys,keypad,outdoor_seating,printer,rain_shower,waterfront,hot_water_kettle,wifi,smart_lock,bath_towel,mini_fridge,convection_oven,terrace,beachfront,free_parking_on_premises,pillow_top_mattress,ceiling_fan,exercise_equipment,childrens_dinnerware,pool,breakfast,dishwasher,bed_linens,self_check_in,hammock,lock_on_bedroom_door,shampoo,double_oven,barbecue_utensils,microwave,other_pets,building_staff,bread_maker,lockbox,shower_gel,fire_extinguisher,paid_parking_off_premises,stove,hair_dryer,24_hour_check_in,ground_floor_access,doorman,smoking_allowed,wheelchair_accessible,accessible_height_bed,dishes_and_silverware,private_entrance,baking_sheet,bathtub,wide_doorway_to_guest_bathroom,heated_towel_rack,shared_pool,espresso_machine,bathtub_with_bath_chair,

In [48]:
total.shape

(16025, 250)

**sentimiento**

In [9]:
# primero bert multilengua

nombre='nlptown/bert-base-multilingual-uncased-sentiment'

modelo=AutoModelForSequenceClassification.from_pretrained(nombre)

tokenizer=AutoTokenizer.from_pretrained(nombre, model_max_length=512, max_lenght=512)

In [10]:
bert_pipe=pipeline(task='sentiment-analysis', model=modelo, tokenizer=tokenizer)

In [11]:
# me falla el número máximo de tokens, a pesar de poner el parámetro. A veces pasa de 512, de ahí esta función.

def extract_sentiment_bert(x: str) -> float:
    try:
        return bert_pipe(x)[0]['score']
    except:
        return 0.5

In [12]:
%%time

total['bert_score']=total.comments.apply(extract_sentiment_bert)

Token indices sequence length is longer than the specified maximum sequence length for this model (670 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (636 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (824 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (588 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (573 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for thi

CPU times: user 1h 28min 4s, sys: 47.7 s, total: 1h 28min 51s
Wall time: 1h 27min 27s


In [13]:
total.head()

,id_x,host_id,host_is_superhost,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,accommodates,bathrooms,bedrooms,beds,price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,x,y,z,property_type_apartment,property_type_barn,property_type_bed_and_breakfast,property_type_boutique_hotel,property_type_bungalow,property_type_camper_rv,property_type_casa_particular_cuba,property_type_cave,property_type_chalet,property_type_condominium,property_type_dome_house,property_type_earth_house,property_type_farm_stay,property_type_guest_suite,property_type_guesthouse,property_type_hostel,property_type_hotel,property_type_house,property_type_hut,property_type_igloo,property_type_loft,property_type_nature_lodge,property_type_other,property_type_pension_south_korea,property_type_serviced_apartment,property_type_tent,property_type_tiny_house,property_type_townhouse,property_type_villa,property_type_yurt,room_type_hotel_room,room_type_private_room,room_type_shared_room,bed_type_couch,bed_type_futon,bed_type_pull_out_sofa,bed_type_real_bed,memory_foam_mattress,beach_essentials,smart_tv,body_soap,day_bed,internet,standing_valet,tv,dryer,toilet,gas_oven,shower_chair,dogs,safety_card,wine_cooler,disabled_parking_spot,step_free_shower,wide_entrance,cooking_basics,formal_dining_area,hbo_go,long_term_stays_allowed,en_suite_bathroom,sun_loungers,beach_view,pocket_wifi,netflix,projector_and_screen,oven,balcony,free_street_parking,ethernet_connection,extra_space_around_bed,accessible_height_toilet,baby_bath,sound_system,patio_or_balcony,high_chair,fixed_grab_bars_for_toilet,first_aid_kit,essentials,toilet_paper,suitable_for_events,wide_entryway,ev_charger,mudroom,coffee_maker,garden_or_backyard,hot_water,family_kid_friendly,private_bathroom,paid_parking_on_premises,changing_table,handheld_shower_head,single_level_home,baby_monitor,wide_clearance_to_shower,washer,touchless_faucets,pets_allowed,crib,other,jetted_tub,steam_oven,host_greets_you,trash_can,laptop_friendly_workspace,private_living_room,buzzer_wireless_intercom,refrigerator,luggage_dropoff_allowed,full_kitchen,smoke_detector,soaking_tub,heated_floors,mobile_hoist,indoor_fireplace,bedroom_comforts,central_air_conditioning,hot_tub,cable_tv,heating,washer___dryer,well_lit_path_to_entrance,pets_live_on_this_property,iron,bathroom_essentials,high_resolution_computer_monitor,hangers,walk_in_shower,elevator,no_stairs_or_steps_to_enter,cats,game_console,extra_pillows_and_blankets,ski_in_ski_out,room_darkening_shades,firm_mattress,fireplace_guards,lake_access,warming_drawer,kitchenette,flat_path_to_guest_entrance,bbq_grill,bidet,air_conditioning,carbon_monoxide_detector,breakfast_table,dvd_player,cleaning_before_checkout,window_guards,kitchen,stair_gates,amazon_echo,outlet_covers,wide_hallways,fixed_grab_bars_for_shower,babysitter_recommendations,table_corner_guards,pack_n_play_travel_crib,gym,wide_entrance_for_guests,childrens_books_and_toys,keypad,outdoor_seating,printer,rain_shower,waterfront,hot_water_kettle,wifi,smart_lock,bath_towel,mini_fridge,convection_oven,terrace,beachfront,free_parking_on_premises,pillow_top_mattress,ceiling_fan,exercise_equipment,childrens_dinnerware,pool,breakfast,dishwasher,bed_linens,self_check_in,hammock,lock_on_bedroom_door,shampoo,double_oven,barbecue_utensils,microwave,other_pets,building_staff,bread_maker,lockbox,shower_gel,fire_extinguisher,paid_parking_off_premises,stove,hair_dryer,24_hour_check_in,ground_floor_access,doorman,smoking_allowed,wheelchair_accessible,accessible_height_bed,dishes_and_silverware,private_entrance,baking_sheet,bathtub,wide_doorway_to_guest_bathroom,heated_towel_rack,shared_pool,espresso_machine,bathtub_with_bath_chair,

In [14]:
# ahora modelo por defecto

pipe=pipeline('sentiment-analysis')

In [15]:
%%time

# Se traduce todo al inglés. Hay comentarios escritos en cirílico, 
# y esos caracteres no son reconocidos por el modelo por defecto.

total.comments=total.comments.apply(lambda x: Translator().translate(x, dest='en').text)

CPU times: user 7min 57s, sys: 26.5 s, total: 8min 23s
Wall time: 47min 49s


In [16]:
def extract_sentiment(x: str) -> float:
    try:
        return pipe(x)[0]['score']
    except:
        return 0.5

In [17]:
%%time

total['def_score']=total.comments.apply(extract_sentiment)

CPU times: user 37min 40s, sys: 20.9 s, total: 38min 1s
Wall time: 45min 3s


In [18]:
total.head()

,id_x,host_id,host_is_superhost,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,accommodates,bathrooms,bedrooms,beds,price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,x,y,z,property_type_apartment,property_type_barn,property_type_bed_and_breakfast,property_type_boutique_hotel,property_type_bungalow,property_type_camper_rv,property_type_casa_particular_cuba,property_type_cave,property_type_chalet,property_type_condominium,property_type_dome_house,property_type_earth_house,property_type_farm_stay,property_type_guest_suite,property_type_guesthouse,property_type_hostel,property_type_hotel,property_type_house,property_type_hut,property_type_igloo,property_type_loft,property_type_nature_lodge,property_type_other,property_type_pension_south_korea,property_type_serviced_apartment,property_type_tent,property_type_tiny_house,property_type_townhouse,property_type_villa,property_type_yurt,room_type_hotel_room,room_type_private_room,room_type_shared_room,bed_type_couch,bed_type_futon,bed_type_pull_out_sofa,bed_type_real_bed,memory_foam_mattress,beach_essentials,smart_tv,body_soap,day_bed,internet,standing_valet,tv,dryer,toilet,gas_oven,shower_chair,dogs,safety_card,wine_cooler,disabled_parking_spot,step_free_shower,wide_entrance,cooking_basics,formal_dining_area,hbo_go,long_term_stays_allowed,en_suite_bathroom,sun_loungers,beach_view,pocket_wifi,netflix,projector_and_screen,oven,balcony,free_street_parking,ethernet_connection,extra_space_around_bed,accessible_height_toilet,baby_bath,sound_system,patio_or_balcony,high_chair,fixed_grab_bars_for_toilet,first_aid_kit,essentials,toilet_paper,suitable_for_events,wide_entryway,ev_charger,mudroom,coffee_maker,garden_or_backyard,hot_water,family_kid_friendly,private_bathroom,paid_parking_on_premises,changing_table,handheld_shower_head,single_level_home,baby_monitor,wide_clearance_to_shower,washer,touchless_faucets,pets_allowed,crib,other,jetted_tub,steam_oven,host_greets_you,trash_can,laptop_friendly_workspace,private_living_room,buzzer_wireless_intercom,refrigerator,luggage_dropoff_allowed,full_kitchen,smoke_detector,soaking_tub,heated_floors,mobile_hoist,indoor_fireplace,bedroom_comforts,central_air_conditioning,hot_tub,cable_tv,heating,washer___dryer,well_lit_path_to_entrance,pets_live_on_this_property,iron,bathroom_essentials,high_resolution_computer_monitor,hangers,walk_in_shower,elevator,no_stairs_or_steps_to_enter,cats,game_console,extra_pillows_and_blankets,ski_in_ski_out,room_darkening_shades,firm_mattress,fireplace_guards,lake_access,warming_drawer,kitchenette,flat_path_to_guest_entrance,bbq_grill,bidet,air_conditioning,carbon_monoxide_detector,breakfast_table,dvd_player,cleaning_before_checkout,window_guards,kitchen,stair_gates,amazon_echo,outlet_covers,wide_hallways,fixed_grab_bars_for_shower,babysitter_recommendations,table_corner_guards,pack_n_play_travel_crib,gym,wide_entrance_for_guests,childrens_books_and_toys,keypad,outdoor_seating,printer,rain_shower,waterfront,hot_water_kettle,wifi,smart_lock,bath_towel,mini_fridge,convection_oven,terrace,beachfront,free_parking_on_premises,pillow_top_mattress,ceiling_fan,exercise_equipment,childrens_dinnerware,pool,breakfast,dishwasher,bed_linens,self_check_in,hammock,lock_on_bedroom_door,shampoo,double_oven,barbecue_utensils,microwave,other_pets,building_staff,bread_maker,lockbox,shower_gel,fire_extinguisher,paid_parking_off_premises,stove,hair_dryer,24_hour_check_in,ground_floor_access,doorman,smoking_allowed,wheelchair_accessible,accessible_height_bed,dishes_and_silverware,private_entrance,baking_sheet,bathtub,wide_doorway_to_guest_bathroom,heated_towel_rack,shared_pool,espresso_machine,bathtub_with_bath_chair,

**Modelado**

In [53]:
COLS=['accommodates', 'air_conditioning', 'availability_30', 'availability_365', 'availability_60', 'availability_90',
      'bathrooms', 'bedrooms', 'beds', 'calculated_host_listings_count', 'calculated_host_listings_count_entire_homes',
      'calculated_host_listings_count_private_rooms', 'calculated_host_listings_count_shared_rooms', 'cleaning_fee',
      'dishwasher', 'extra_people', 'guests_included','latitude', 'longitude', 'maximum_nights', 'minimum_nights', 
      'number_of_reviews', 'number_of_reviews_ltm', 'room_type_private_room', 'room_type_shared_room', 'security_deposit',
      'price', 'bert_score', 'def_score']

len(COLS)

27

In [54]:
total=total[COLS]

In [32]:
ctr=CTR(verbose=0)

In [33]:
# todo

X=total.drop('price', axis=1)
y=total.price

X_train, X_test, y_train, y_test = tts(X, y, train_size=0.8, test_size=0.2, random_state=42)

ctr.fit(X_train, y_train)

y_pred=ctr.predict(X_train)

print(f'Train RMSE: {mse(y_train, y_pred, squared=False)}')
print(f'Train MAE: {mae(y_train, y_pred)}')
print(f'Train R2: {r2(y_train, y_pred)}')
print()

y_pred=ctr.predict(X_test)

print(f'Test RMSE: {mse(y_test, y_pred, squared=False)}')
print(f'Test MAE: {mae(y_test, y_pred)}')
print(f'Test R2: {r2(y_test, y_pred)}')

Train RMSE: 78.41835672801739
Train MAE: 31.7541622615005
Train R2: 0.9426352209249058

Test RMSE: 242.7643930141899
Test MAE: 51.77057236108888
Test R2: 0.45370577452020644


In [35]:
# bert

X=total.drop(columns=['price', 'def_score'])
y=total.price

X_train, X_test, y_train, y_test = tts(X, y, train_size=0.8, test_size=0.2, random_state=42)

ctr.fit(X_train, y_train)

y_pred=ctr.predict(X_train)

print(f'Train RMSE: {mse(y_train, y_pred, squared=False)}')
print(f'Train MAE: {mae(y_train, y_pred)}')
print(f'Train R2: {r2(y_train, y_pred)}')
print()
y_pred=ctr.predict(X_test)

print(f'Test RMSE: {mse(y_test, y_pred, squared=False)}')
print(f'Test MAE: {mae(y_test, y_pred)}')
print(f'Test R2: {r2(y_test, y_pred)}')

Train RMSE: 78.77325616430595
Train MAE: 31.562292321648872
Train R2: 0.9421148122464515

Test RMSE: 252.71204936015997
Test MAE: 53.096926041533365
Test R2: 0.4080179563715338


In [37]:
# default

X=total.drop(columns=['price', 'bert_score'])
y=total.price

X_train, X_test, y_train, y_test=tts(X, y, train_size=0.8, test_size=0.2, random_state=42)

ctr.fit(X_train, y_train)

y_pred=ctr.predict(X_train)

print(f'Train RMSE: {mse(y_train, y_pred, squared=False)}')
print(f'Train MAE: {mae(y_train, y_pred)}')
print(f'Train R2: {r2(y_train, y_pred)}')
print()
y_pred=ctr.predict(X_test)

print(f'Test RMSE: {mse(y_test, y_pred, squared=False)}')
print(f'Test MAE: {mae(y_test, y_pred)}')
print(f'Test R2: {r2(y_test, y_pred)}')

Train RMSE: 80.39094951289148
Train MAE: 31.627514327170356
Train R2: 0.9397129314122158

Test RMSE: 252.0876870910165
Test MAE: 54.0183243873366
Test R2: 0.410939500188058


In [57]:
# el que ya tenia

X=total.drop(columns=['price', 'bert_score', 'def_score'])
y=total.price

X_train, X_test, y_train, y_test=tts(X, y, train_size=0.8, test_size=0.2, random_state=42)

ctr.fit(X_train, y_train)

y_pred=ctr.predict(X_train)

print(f'Train RMSE: {mse(y_train, y_pred, squared=False)}')
print(f'Train MAE: {mae(y_train, y_pred)}')
print(f'Train R2: {r2(y_train, y_pred)}')
print()
y_pred=ctr.predict(X_test)

print(f'Test RMSE: {mse(y_test, y_pred, squared=False)}')
print(f'Test MAE: {mae(y_test, y_pred)}')
print(f'Test R2: {r2(y_test, y_pred)}')

Train RMSE: 15.234506986801547
Train MAE: 10.708168512526674
Train R2: 0.8379971953000285

Test RMSE: 19.35887907614787
Test MAE: 13.315279722680128
Test R2: 0.7356717260299546


Con esta transformación, lo único que he conseguido es sobreajustar el modelo. Es verdad que solo he usado la primera review, debería de juntar todo el texto y volver a entrenarlo todo.